# Impulsive BCR4BP problem

In [180]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
plotly()

Plots.PlotlyBackend()

In [217]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [218]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [219]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0


1

In [220]:
res.x0

6-element Vector{Float64}:
  1.1226879860196828
  0.0
 -5.228885624166955e-26
  0.0
  0.1650993133797659
  0.0

In [221]:
x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1)
eps = 1e-6
if ys0[1] > 0
    svf = vcat(res.x0 + eps*ys0, 1.0)
else
    svf = vcat(res.x0 - eps*ys0, 1.0)
end

Linear stability ν = 618.7618470919092


7-element Vector{Float64}:
  1.1226882532375977
  2.793813746152613e-7
 -5.228883154550508e-26
 -7.710131455619898e-7
  0.16509880733691906
  1.339504158679477e-31
  1.0

In [222]:
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

### Construct optimization problem

In [223]:
LPOArrival.period
param3b.as

388.8212386592645

In [237]:
propagate_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false) where T
    # unpack
    βf, tof, eta, sma, ecc, raan, ϕ = x  # βf: Sun angle at final time
    tof_fwd = tof * eta
    tof_bck = tof * (1 - eta)
    
    # construct initial state
    sv0_kep = [sma, ecc, 0.0, raan, 0.0, 0.0]
    β0 = βf - param3b.oms*(tof_fwd + tof_bck)   # initial Sun angle
    θf = π - βf  # terminal moon angle 
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1) 
    sv0 = vcat(inertial2rotating(sv0_i, 0.0, 1.0)/param3b.as + [-param3b.mu2/param3b.as + param3b.mus/ (param3b.mus+1) ,0,0,0,0,0], 1.0)
    println(sv0)

    # construct final state
    #x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))
    #tspan = [0, ϕ*LPOArrival.period]
    #prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, tspan, [param3b.mu2,])
    #sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)
    svf = vcat(SailorMoon.set_terminal_state(ϕ, θf, param3b, LPOArrival), 1.0)
    
    # forward propagation
    params = [param3b.mu2, param3b.mus, param3b.as, β0, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
    tspan = [0, tof_fwd]
    prob_fwd = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame!, sv0, tspan, params)
    sol_fwd = solve(prob_fwd, Tsit5(), reltol=1e-12, abstol=1e-12);
    
    println(svf, "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    # back propagation
    params = [param3b.mu2, param3b.mus, param3b.as, βf, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
    tspan = [0, -tof_bck]
    prob_bck = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame!, svf, tspan, params)
    sol_bck = solve(prob_bck, Tsit5(), reltol=1e-12, abstol=1e-12);
    
    # residual
    if get_sols == false
        return sol_bck.u[end][1:3] - sol_fwd[end][1:3]
    else
        return sol_fwd, sol_bck
    end
end

#45 (generic function with 2 methods)

In [238]:
# θf, tof, eta, sma, ecc, raan, ϕ = x
xtest = [
    2.4, 22, 0.4, 1.8, 0.87, 4.5, 0.02
]

lx = [
    2.6, 18, 0.3, 1.9, 0.7, 0.0, -1.0
]
ux = [
    3.2, 27, 0.7, 2.4, 0.995, 2π, 1.0
]

7-element Vector{Float64}:
  3.2
 27.0
  0.7
  2.4
  0.995
  6.283185307179586
  1.0

In [239]:
sol_fwd, sol_bck = propagate_trajectory(xtest, true);
propagate_trajectory(xtest, false)

[0.999838848865133, -0.0005882961751842113, 0.0, 0.006475504165415955, -0.0013963856996310512, 0.0, 1.0]
0.9998420623620815 1.0006068742571201 0.9988303945966438
0.9998420623620815 1.0006068742571201 0.9988303945966438
1.0031643994661192 1.001315342180079 1.000593967373769
0.9998466561801898 1.0006078491079453 0.9988325999292663
0.9998513892540676 1.0006088678177583 0.9988348881406157
0.9998677165527446 1.0006124249146515 0.9988428336217992
0.9998699952049811 1.000612927925024 0.998843950293794
0.999870563890508 1.0006130537558968 0.9988442293249143
0.9998705637474891 1.0006130549634447 0.9988442305321917
0.999881754215665 1.0006155338248828 0.998849726556464
0.9998932717160037 1.0006181704561503 0.9988554788288849
0.9999329645190053 1.000627512540349 0.9988756128350084
0.9999384977839604 1.0006288537973937 0.9988784658872877
0.9999398784911604 1.000629190245897 0.9988791798564074
0.9999398787230432 1.0006291985440023 0.9988791881549268
0.9999544026657834 1.0006327473620193 0.998886712

1.003706511597238 1.0060465765689686 1.006253013223681
1.0037075942906355 1.0060532441246697 1.0062614286363254
1.003707574457153 1.0060534258705924 1.0062616103266493
1.0037162867380687 1.0061073423822555 1.0063296214197872
1.0037247507403768 1.0061631434604392 1.0063999463184092
1.0037524065487042 1.0063562227252785 1.0066430927578898
1.0037560232642535 1.006383215352786 1.0066770687627866
1.0037569156104222 1.0063899585501876 1.0066855546477496
1.0037568923757783 1.0063901390150671 1.006685735048467
1.0037640660329907 1.0064446583133704 1.006754304234156
1.0037709394518661 1.0065010427119878 1.0068251635956753
1.0037930957237775 1.0066960195054009 1.0070700173222553
1.0037959437695692 1.0067232589037234 1.0071042108687573
1.0037966441420056 1.0067300629282605 1.0071127501834303
1.003796617522558 1.0067302418111388 1.0071129289937903
1.0038022398432078 1.0067852446365948 1.0071819192023954
1.0038075103596316 1.006842089031774 1.0072531713537913
1.0038241269145354 1.00703853687474 1.0

1.0016972939534523 1.0142347217509202 1.0164077699883236
1.0016656390428902 1.0142648664884761 1.0164456174715861
1.0015553137835158 1.0143665436636047 1.0165733499709206
1.0015397586713988 1.0143803456952325 1.0165907247353634
1.001535867905014 1.0143837763870251 1.0165950446412395
1.0015357786195733 1.0143838350213763 1.0165951031588576
1.0015043510696142 1.0144115162618281 1.0166299138883108
1.0014715872174178 1.0144392757276286 1.0166649547045274
1.0013574777469292 1.0145326446698677 1.0167829140501228
1.0013414014474862 1.0145452760513622 1.0167989123202377
1.0013373808582509 1.014548413965293 1.0168028880222892
1.0013372906034839 1.014548466051571 1.0168029399975305
1.0013048148254782 1.0145737836168014 1.0168349721772052
1.001270984695402 1.0145990804524487 1.0168671213555118
1.0011532395930758 1.0146838772547628 1.0169750179391979
1.0011366630351322 1.0146953016462876 1.0169895998567624
1.001132517857101 1.0146981377309454 1.0169932214281192
1.0011324267514505 1.014698183210441

0.9940774524784294 1.0116072381161831 1.013887611639719
0.9940573040826445 1.0115767214048144 1.0138538240628017
0.9940522779323503 1.0115690829206245 1.013845366490261
0.9940522113415506 1.0115689455467172 1.013845229194239
0.9940116567497934 1.011507264625688 1.013776907474961
0.9939700366260324 1.0114426548391995 1.0137053780353238
0.9938270549602052 1.0112166689310857 1.013454944415241
0.9938072117008269 1.011184667477786 1.013419459470047
0.9938022621244657 1.0111766586289541 1.0134105785094074
0.9938021982221704 1.0111765161605686 1.0134104361206737
0.9937622693187999 1.011111856346101 1.013338709927989
0.9937213132265003 1.0110441909111298 1.0132636873755079
0.993580681036911 1.0108077106373727 1.0130012601082938
0.9935611742793639 1.010774253379703 1.0129641113129693
0.9935563091003768 1.0107658814253224 1.0129548153704153
0.9935562480019063 1.0107657341130345 1.0129546681389188
0.9935170080293283 1.0106981555349737 1.0128796070883341
0.9934767807891914 1.010627498150619 1.0128

0.9895195944042959 0.9940556532457433 0.9942464685549114
0.9895191569568621 0.9940418109063807 0.9942309269893679
0.9895191858101822 0.9940416455328427 0.9942307615490119
0.9895156980768208 0.9939300358268722 0.9941054629652446
0.989512599695207 0.9938148675324389 0.9939761741072723
0.9895034686954423 0.9934172097544973 0.9935297355309209
0.989502432818353 0.9933616964249147 0.9934674051342072
0.9895021845939281 0.9933478369161578 0.9934518439316604
0.9895022161879312 0.9933476741011318 0.9934516810427397
0.9895002542118213 0.9932359122191786 0.9933262097682222
0.9894987295738283 0.9931206233528852 0.9931967817188148
0.9894950299878098 0.9927226583882852 0.9927499914841333
0.9894947518036825 0.9926671189916619 0.9926876312551964
0.9894946927362009 0.992653253717122 0.9926720634403969
0.9894947270227468 0.9926530935985252 0.9926719032409848
0.9894942906643481 0.9925412721964674 0.9925463650746938
0.9894943390958801 0.9924259582575382 0.9924169068861397
0.9894960669042734 0.9920280155257

...

0.9928950361192068 0.9752721455381335 0.9732041869855992
0.9929318224842101 0.9751962050937358 0.973119789426453
0.9929700293675292 0.9751188099943247 0.9730337410673862
0.9931027031672096 0.9748544658531114 0.9727400526662829
0.9931213340344681 0.9748180047494726 0.9726995585636484
0.9931259907152064 0.9748089216501038 0.9726894713270945
0.993126081053019 0.974808900061691 0.9726894496045335
0.9931637409888863 0.9747355828815654 0.9726080640904959
0.9932028321372617 0.9746609160115984 0.9725251467342317
0.9933385049578458 0.9744060547790224 0.9722423461142508
0.9933575460353513 0.9743709267738228 0.9722033830981325
0.9933623047806402 0.9743621769450822 0.9721936786179247
0.993362395822281 0.9743621622719506 0.9721936638160757
0.9934008751412483 0.9742915487886067 0.9721153832998868
0.993440794146175 0.9742196917123409 0.9720356913325432
0.9935792697909489 0.9739745906233788 0.9717640993019718
0.9935986934761172 0.9739408337178868 0.9717267113733496
0.9936035473845359 0.973932426618821

1.0006357943657276 0.9696322645800224 0.9672787563439885
1.000763331144408 0.9696930152673081 0.9673604520428112
1.000780979888346 0.9697018829640629 0.9673723173717077
1.0007853798376298 0.9697041157836024 0.9673753011632261
1.000785429163158 0.9697042754583173 0.9673754608988433
1.000820917515562 0.9697223986303529 0.9673996760635822
1.0008572047681712 0.9697419733950473 0.9674256065535143
1.0009814822688745 0.9698121162812844 0.9675182680826602
1.0009986691248136 0.9698222833582767 0.9675316512960327
1.0010029534425375 0.9698248401050027 0.9675350135043361
1.0010029998171177 0.9698250032658325 0.9675351767269766
1.0010375558902598 0.9698457400752073 0.9675624442758354
1.001072866140486 0.9698679867109294 0.9675914976393988
1.0011937262567352 0.9699472781390923 0.9676948537962908
1.0012104293706958 0.9699587101149502 0.9677097164886533
1.0012145926095923 0.969961582158823 0.96771344750129
1.001214635975975 0.9699617485970184 0.9677136140015665
1.001248215706766 0.9699850295830309 0.9

Excessive output truncated after 524302 bytes.


1.003417424094648 0.9732793982891461 0.9718658059527879
1.0034174208952271 0.9732795871958426 0.9718659948617763
1.003433406923536 0.9733296351113309 0.9719279634690624
1.003449392597636 0.973381645461717 0.9719923085577139
1.0035030613223215 0.97356223963193 0.9722158212886962
1.003510314746932 0.9735875829843853 0.9722472000434899
1.0035121154023499 0.973593918493257 0.9722550435550648
1.003512108691785 0.9735941071962146 0.9722552322522373
1.003526615318025 0.9736453300443569 0.9723186362964488
1.0035410684824186 0.9736985151003053 0.972384422228688
1.0035894293134069 0.9738830513855091 0.9726127766609185
1.0035959394344662 0.9739089272924165 0.9726448112383411
1.0035975543885172 0.9739153949990308 0.9726528176243515
1.003597544164826 0.9739155832373337 0.9726530058484495
1.0036105508360105 0.973967870434614 0.972717720108551
1.003623450980761 0.9740221156276156 0.9727848171750797
1.003666435169008 0.9742101971506165 0.9730175637856603
1.0036721927159389 0.9742365500247978 0.973050

3-element Vector{Float64}:
 23.466859851796926
 -8.476616396510535
  7.706136834653387e-6

In [244]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
# scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
plot!(pcart, Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
# plot!(pcart, Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=:purple, linewidth=1.5, label="bck")
plot!(pcart; title="Initial guess")


t = range(0,stop=2*pi,length=360)
re = param3b.mu2 / param3b.as * [cos.(t), sin.(t)]
rm = param3b.mu1 / param3b.as * [cos.(t), sin.(t)]
plot!(pcart, Array(re)[1,:] + param3b.mus / (param3b.mus+1), Array(re)[2,:], color=:blue, linewidth=1.5, label="fwd")


    
pcart

LoadError: MethodError: no method matching +(::Vector{Vector{Float64}}, ::Float64)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at C:\Users\yujit\AppData\Local\Programs\Julia-1.7.3\share\julia\base\operators.jl:655
[0m  +([91m::Union{VectorizationBase.AbstractMask{W}, VectorizationBase.VecUnroll{<:Any, W, SIMDTypes.Bit, <:VectorizationBase.AbstractMask{W}}} where W[39m, ::Union{Bool, Float16, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, SIMDTypes.Bit}) at C:\Users\yujit\.julia\packages\VectorizationBase\YKf7m\src\base_defs.jl:348
[0m  +([91m::AbstractGray{Bool}[39m, ::Number) at C:\Users\yujit\.julia\packages\ColorVectorSpace\bhkoO\src\ColorVectorSpace.jl:337
[0m  ...

In [ ]:
# problem settings
ng = 3
lg = [0.0 for idx=1:ng];
ug = [0.0 for idx=1:ng];

fitness! = function (g, x)
    # evaluate objective & objective gradient (trivial)
    #f = 1       # whichever x corresponds to e.g. mass at LEO
    #g[:] = propagate_trajectory(x, false)
    
    sol_fwd, sol_bck = propagate_trajectory(x, true)
    g[:] = sol_bck.u[end][1:3] - sol_fwd.u[end][1:3]
    f = norm(sol_bck.u[end][4:6] - sol_fwd.u[end][4:6])
    return f
end

In [ ]:
gfoo  = zeros(ng)
fitness!(gfoo, xtest)
gfoo

In [ ]:
ip_options = Dict(
    "max_iter" => 100,   # approx 100
    "print_level" => 4,
    "acceptable_tol" => 1e-6,
    "constr_viol_tol" => 1e-6,
)
x0 = [el for el in xtest]
xopt, fopt, info = joptimise.minimize(
    fitness!, x0, ng;
    lx=lx, ux=ux, lg=lg, ug=ug,
    #derivatives=joptimise.ForwardAD(),
    options=ip_options,
);

In [ ]:
info

In [ ]:
sol_fwd, sol_bck = propagate_trajectory(xopt, true);
propagate_trajectory(xopt, false)

In [ ]:
dr = sol_bck.u[end][1:3] - sol_fwd.u[end][1:3]

In [ ]:
dv = sol_bck.u[end][4:6] - sol_fwd.u[end][4:6]

In [ ]:
norm(dv)*param3b.lstar/param3b.tstar  # km/s

In [ ]:
for i = 1:length(xopt)
    @printf("% 3.3f | % 3.3f | % 3.3f\n", lx[i], xopt[i], ux[i])
end

In [ ]:
lx

In [ ]:
ux

In [ ]:
plotly()

In [ ]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

In [ ]:
# unpack optimal solution
βf, tof, eta, sma, ecc, raan, ϕ = xopt
tof_fwd = tof * eta
tof_bck = tof * (1 - eta)
β0 = βf - param3b.oms*tof                # initial Sun angle

In [ ]:
tof

## Plot in Sun-Earth rotating frame

In [ ]:
SE_system = R3BP.get_cr3bp_param(10,399)
EM_system = R3BP.get_cr3bp_param(399,301)

scale_l = EM_system.lstar/SE_system.lstar
scale_v = (EM_system.lstar/EM_system.tstar) / (SE_system.lstar/SE_system.tstar)
scale_t = EM_system.tstar/SE_system.tstar

In [ ]:
function planetmoon2sunplanet(
    state::Vector, μ_moon::Real, μ_sun::Real, theta::Real, om::Real,
    scale_l::Real, scale_v::Real, scale_t::Real, final_shift::Bool=false
)
    # 1. shift to Planet-centered, planet-moon rotating frame
    state_1 = state + [μ_moon, 0, 0, 0, 0, 0]
    # 2. convert to PLanet-centered, inertial
    state_2 = rotating2inertial(state_1, theta, om)
    # 3. re-scale
    state_3 = vcat(
        state_2[1:3] * scale_l,
        state_2[4:6] * scale_v
    )
    # 4. convert to Planet-centered, Sun-planet rotating frame
    theta_sp = scale_t * theta
    state_4 = inertial2rotating(state_3, theta_sp, om)
    # 5. shift to Sun-planet barycenter centered, Sun-planet rotating frame
    if final_shift == true
        return state_4 + [1 - μ_sun, 0, 0, 0, 0, 0]
    else
        return state_4
    end
end


In [ ]:
sol_bck_se = zeros(7,length(sol_bck.u))
for (idx,sv) in enumerate(sol_bck.u)
    theta = βf + param3b.oml*sol_bck.t[idx]
    sol_bck_se[1:6,idx] = planetmoon2sunplanet(
        sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
        scale_l, scale_v, scale_t
    )
    sol_bck_se[7,idx] = sv[7]
end

sol_fwd_se = zeros(7,length(sol_fwd.u))
for (idx,sv) in enumerate(sol_fwd.u)
    #theta = β0 + sol_fwd.t[idx]
    theta = βf + param3b.oml*sol_bck.t[end] - param3b.oml*(tof_fwd - sol_fwd.t[idx])
    sol_fwd_se[1:6,idx] = planetmoon2sunplanet(
        sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
        scale_l, scale_v, scale_t
    )
    sol_fwd_se[7,idx] = sv[7]
end

In [ ]:
βf, β0

In [ ]:
βf - param3b.oms*tof

In [ ]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

# trajectory
#plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_fwd_se[1,:], sol_fwd_se[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_bck_se[1,:], sol_bck_se[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

## Plot in inertial frame

In [ ]:
sol_fwd.t[end] - sol_bck.t[end]

In [ ]:
sol_fwd_i = zeros(7,length(sol_fwd.u))
for (idx,sv) in enumerate(sol_fwd.u)
    theta = βf - tof + sol_fwd.t[idx]
    sol_fwd_i[1:6,idx] = rotating2inertial(sv[1:6], theta, 1.0)
    sol_fwd_i[7,idx] = sv[7]
end

sol_bck_i = zeros(7,length(sol_bck.u))
for (idx,sv) in enumerate(sol_bck.u)
    theta = βf + sol_bck.t[idx]
    sol_bck_i[1:6,idx] = rotating2inertial(sv[1:6], theta, 1.0)
    sol_bck_i[7,idx] = sv[7]
end

In [ ]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

# trajectory
plot!(sol_fwd_i[1,:], sol_fwd_i[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_bck_i[1,:], sol_bck_i[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

In [ ]:
1737/param3b.lstar

In [ ]:
0.001*param3b.lstar